In [2]:
import os
import numpy    as np
import pandas   as pd
import psycopg2 as psy

from datetime import datetime as dt
from random   import randrange
from random   import sample
from random   import choice

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
#''' Check if file exists on path. Move existing file to ARCH. Create new input for office_dim.'''


In [73]:
# Setup script input parameters

min_obs = 5                      # minimal number of observation in output dataset
max_obs = 20                     # maximal number of observation in output dataset

min_age = 18                     # minimal age of client
max_age = 90                     # maximal age of client

min_street_number = 1            # minimal street number
max_street_number = 25           # maximal street number

min_local_number = 1            # minimal street number
max_local_number = 100          # maximal street number

obs = randrange(min_obs,max_obs) # number of observation
obs

10

In [6]:
path_to_arch = "/Users/szymonbocian/Documents/Python/Arch/office"
path_to_dir  = "/Users/szymonbocian/Documents/Python/input"
file_name    = "office_dim.csv"

now          = dt.now()
path_to_file = os.path.join(path_to_dir, file_name)

if not(os.path.exists(path_to_file) and os.path.isdir(path_to_arch)):
    raise Exception('Incorrect dir')

time_stamp = "_" + str(now)[:10]
arch_file_name = 'arch_' + file_name.replace(".csv",time_stamp + ".csv")
 
os.rename(path_to_file, os.path.join(path_to_arch, arch_file_name))

In [74]:
# connection to database and return list of column column_list
try:
    connection = psy.connect( user = "szymonbocian",
                              password = "",
                              host = "localhost",
                              port = "5432",
                              database = "dwh_call_center")

    cursor = connection.cursor()

    query = "SELECT column_name FROM INFORMATION_SCHEMA.columns WHERE table_schema = 'stg' AND table_name = 'office_dim';"
    cursor.execute(query)
    column_list = cursor.fetchall()
    
    query = "SELECT DISTINCT office_key FROM pro.office_dim;"
    cursor.execute(query)
    office_list = cursor.fetchall()
    
    query = "SELECT office_key FROM pro.office_dim WHERE office_hier_key IS NULL;"
    cursor.execute(query)
    office_hier_list = cursor.fetchall()
    
except (Exception, psy.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
        if(connection):
            cursor.close()
            connection.close()

column_list = [''.join(c for c in s if c not in '(),') for s in column_list]

a = list()
for row in office_list:
    a.append(row[0])

office_list = a

a = list()
for row in office_hier_list:
    a.append(row[0])

office_hier_list = a

print(column_list)
print(office_list)
print(office_hier_list)

['old_office_key', 'office_hier_key', 'office_name', 'address_street_name', 'address_street_number', 'address_local_number', 'location_latitude', 'location_longitude', 'office_business_key']
[1]
[1]


In [75]:
A = ['Aleppo','Bern','Cincinnati','Dobrzyń','Erevan','Fez','Grodno','Hasting','Irkuck','Jastarnia','Kohln','Lublin','Monaco','Neuestadt','Okocim','Paris']
B = ['Adalbert','Bismark','Chopin','Dionizos','Einstein','Foch','Garibaldi','Himilsbach','Ismail','Jelcyn','Kirsh','Lang','Mickiewicz','Nobel','Otto','Preslay']

In [81]:
df = pd.DataFrame({
    "oldOfficeKey" : np.random.choice([choice(office_list), ""], obs, True, [0.025, 0.975])
})

df['officeHierKey']       = np.random.choice(office_hier_list)
df['officeName']          = sample(["Call Center in " + a + " city" for a in A], k = obs)
df['addressStreetName']   = sample([b + "'s st." for b in B], k = obs)
df['addressStreetNumber'] = pd.Series(np.round(np.random.uniform(min_street_number,max_street_number,obs),0)).astype(int)
df['addressLocalNumber']  = pd.Series(np.round(np.random.uniform(min_local_number,max_local_number,obs),0)).astype(int)
df['locationLatitude']    = np.round(np.random.uniform(-90,90,obs),4)
df['locationLongitude']   = np.round(np.random.uniform(-180,180,obs),4)

df

,oldOfficeKey,officeHierKey,officeName,addressStreetName,addressStreetNumber,addressLocalNumber,locationLatitude,locationLongitude
0,,1,Call Center in Bern city,Ismail's st.,6,4,-70.3277,-124.6332
1,1,1,Call Center in Cincinnati city,Mickiewicz's st.,9,54,-2.9987,163.3078
2,,1,Call Center in Hasting city,Himilsbach's st.,23,90,17.8657,-39.4457
3,,1,Call Center in Irkuck city,Chopin's st.,21,93,-77.2068,-39.1496
4,,1,Call Center in Fez city,Garibaldi's st.,13,40,35.1367,-59.0527
5,,1,Call Center in Okocim city,Adalbert's st.,2,84,-50.5175,82.3411
6,,1,Call Center in Jastarnia city,Einstein's st.,2,41,-39.7994,8.1631
7,,1,Call Center in Grodno city,Preslay's st.,12,11,-9.9801,-75.2828
8,,1,Call Center in Dobrzyń city,Nobel's st.,11,77,-53.4983,-56.8564
9,,1,Call Center in Monaco city,Foch's st.,17,32,25.2470,-95.4653


In [82]:
# Create office business key
on  = df.officeName.map(str).str.upper()
asn = df.addressStreetName.map(str).str.upper()
sn  = ('0' + (df.addressStreetNumber).astype(str))
ln  = ('0' + (df.addressLocalNumber).astype(str))

df['officeBusinessKey'] = on.str.slice(0,3) + asn.str.slice(0,3) + [s[-2:] for s in sn] + [l[-2:] for l in ln]
df

,oldOfficeKey,officeHierKey,officeName,addressStreetName,addressStreetNumber,addressLocalNumber,locationLatitude,locationLongitude,officeBusinessKey
0,,1,Call Center in Bern city,Ismail's st.,6,4,-70.3277,-124.6332,CALISM0604
1,1,1,Call Center in Cincinnati city,Mickiewicz's st.,9,54,-2.9987,163.3078,CALMIC0954
2,,1,Call Center in Hasting city,Himilsbach's st.,23,90,17.8657,-39.4457,CALHIM2390
3,,1,Call Center in Irkuck city,Chopin's st.,21,93,-77.2068,-39.1496,CALCHO2193
4,,1,Call Center in Fez city,Garibaldi's st.,13,40,35.1367,-59.0527,CALGAR1340
5,,1,Call Center in Okocim city,Adalbert's st.,2,84,-50.5175,82.3411,CALADA0284
6,,1,Call Center in Jastarnia city,Einstein's st.,2,41,-39.7994,8.1631,CALEIN0241
7,,1,Call Center in Grodno city,Preslay's st.,12,11,-9.9801,-75.2828,CALPRE1211
8,,1,Call Center in Dobrzyń city,Nobel's st.,11,77,-53.4983,-56.8564,CALNOB1177
9,,1,Call Center in Monaco city,Foch's st.,17,32,25.2470,-95.4653,CALFOC1732


In [83]:
# Create csv file with given destination

df.columns = column_list
df.to_csv(path_to_file, index = None, header = True)